# TUG Test Model training

## Imports

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers

from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata as _metadata

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.utils import shuffle

import pandas as pd
import numpy as np

## Global variables

In [ ]:
FEATURES_PATH = "FEATURES"
MODEL_PATH = "MODEL"
NUM_CLASSES = 5

## Data loading

In [ ]:
data_wear = pd.read_csv(os.path.join(FEATURES_PATH,'features_full_norm_01.csv'))

X_norm = data_wear.drop(columns=["CLASS"])
Y_norm = pd.get_dummies(data_wear["CLASS"])

data_wear["CLASS"].value_counts()

## Write class labels to file

In [ ]:
with open(os.path.join(MODEL_PATH,'labels.txt'), 'w') as labels_file:
    labels = Y_norm.columns.values
    for i, label in enumerate(labels):
        labels_file.write(label)
        if i != len(labels) - 1:
            labels_file.write("\n");

## Data split

In [ ]:
X_train_norm, X_test_norm, Y_train_norm, Y_test_norm = train_test_split(X_norm, Y_norm, test_size = 0.3, random_state=53, stratify=Y_norm)
X_train_norm, X_val_norm, Y_train_norm, Y_val_norm = train_test_split(X_train_norm, Y_train_norm, test_size = 0.2, random_state=53, stratify=Y_train_norm)

## Model

### Architecture definition

In [ ]:
def create_model():
    model = keras.Sequential(
        [
            layers.Dense(256, input_shape=(47,), use_bias=False),
            layers.BatchNormalization(),
            layers.Activation('relu'),
            layers.Dense(256, use_bias=False),
            layers.BatchNormalization(),
            layers.Activation('relu'),
            layers.Dense(NUM_CLASSES, activation='softmax')
        ]
    )
    model.summary()

    opt = optimizers.Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

### Training

In [ ]:
model_norm = create_model()
model_norm.fit(X_train_norm, Y_train_norm, batch_size=50, epochs=50, validation_data=(X_val_norm,Y_val_norm))

### Evaluation

In [ ]:
Y_pred = model_norm.predict(X_test_norm)
Y_pred_lab = np.argmax(Y_pred, axis=1)
Y_test_lab = np.argmax(Y_test_norm.to_numpy(), axis=1)
print("Training results:")
print("\tAccuracy: {0}".format(accuracy_score(Y_test_lab, Y_pred_lab)))
print("\tConfusion matrix:")
print(confusion_matrix(Y_test_lab, Y_pred_lab))
print("\tClassification report:")
print(classification_report(Y_test_lab, Y_pred_lab, target_names=Y_norm.columns))

## TensorFlow Lite Conversion

### Save model as TensorFlow Lite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_norm)
tflite_model = converter.convert()

with open(os.path.join(MODEL_PATH, 'model_full_norm.tflite'), 'wb') as file:
    file.write(tflite_model)

### Load saved model and add class labels as metadata

In [ ]:
model_buffer = writer_utils.load_file(os.path.join(MODEL_PATH, 'model_full_norm.tflite'))

populator = _metadata.MetadataPopulator.with_model_buffer(model_buffer)
populator.load_associated_files([os.path.join(MODEL_PATH, 'labels.txt')])
populator.populate()
updated_model_buff = populator.get_model_buffer()
with open(os.path.join(MODEL_PATH, 'model_full_norm.tflite'), "wb") as f:
    f.write(updated_model_buff)